# Aim

Get FFAST working at a basic level (no noise, no window_var, etc.).

- for reasonably high-snr like around 5db (without the bin-length gain) look at kay vs. kay2 my guess is kay will fail but kay2 will keep working.  how much db can you go lower with kay2. is kay2 alway better?
- how does all this compare to "new" method?
- keep an eye on the total number of samples used by each method.

In [1]:
%load_ext autoreload
%autoreload 2

from ffast_python import *
from matplotlib import pyplot as plt
import numpy as np
import time

from tqdm import tqdm_notebook as tqdm

- making the seed of delays 0 makes the non-singleton locations pass the singleton test wrongly

In [49]:
7**3

343

In [51]:
19**2

361

In [33]:
params = make_args()

params.primes = [7, 19]
params.prime_powers = [3, 2]

params.bins = [i**j for i,j in zip(params.primes, params.prime_powers)]
print(params.bins)

params.length = np.prod(params.bins)
print(np.log2(params.length))

params.iterations = 50
params.snr = 3 - 10*np.log10(params.bins[1])
# params.snr = -22

print('snr: {}'.format(params.snr))

params.sparsity = 1

# for ml
# params.bin_processing_method = 'ml'
# params.chains = 500
# params.delays = 1

# for kay
# params.bin_processing_method = 'kay'
params.bin_processing_method = 'kay2'
params.chains = 16 # needs to be int(np.floor(np.log2(params.length))) [rounds down]
params.delays = 3 # parameter to play with

# for new
# params.bin_processing_method = 'new'
# params.chains = 1 # this is set to 1 always
# params.delays = 1 # parameter to play with [the larger this value the larger the gap should be]



config = Config(params)
config.compute_params()

input_signal = ExperimentInputSignal(config)
input_signal.process()
output_signal = ExperimentOutputSignal(config, input_signal)

[343, 361]
16.917919793059983
snr: -22.575072019056577


In [34]:
np.sum(np.array(params.primes)*np.array(params.prime_powers))*params.delays

177

In [35]:
params.chains*params.delays

48

In [36]:
config.get_noise_sd()**2

180.9285913394453

In [37]:
# vars(config)

In [38]:
config.display()

Running experiment mode (for now)
Signal length: 123823
Signal sparsity: 1
Signal-to-noise ratio (dB): -22
Random phase
Delays: 48
Bins: 343 361
Bin processor: kay2


In [39]:
# plt.plot(np.real(input_signal.time_signal))
# plt.plot(np.imag(input_signal.time_signal))
# plt.show()

In [40]:
# len(input_signal.time_signal)

In [41]:
# signal_fft = np.fft.fft(input_signal.time_signal)
# plt.plot(np.abs(signal_fft))
# plt.title('loc: {}'.format(input_signal.freqs))
# plt.show()

In [42]:
# input_signal.time_signal *= 2 # demonstrating that the input signal can be changed

In [43]:
ffast = FFAST(config, input_signal, output_signal)

In [44]:
ffast.frontend.process()
print(ffast.frontend.get_used_samples_nb()/ config.signal_length)

0.24177253014383435


In [45]:
params.bins

[343, 361]

In [46]:
ffast = FFAST(config, input_signal, output_signal)

if not config.help_displayed:
    # this disregards the time to make the signal
    time_initial = int(np.round(time.time() * 1000)) 

    iterations = config.iterations
    
    for i in tqdm(range(iterations)):
        input_signal.process() # ffast.get_delays() in the CPP code as an argument
        print('input freq: {}'.format(input_signal.freqs))
        
        print('true singleton bins:')
        for stage_idx, bin_size in enumerate(params.bins):
            print('    stage {} bin {}'.format(stage_idx, input_signal.freqs[0] % bin_size))
        
        ffast.process()
        output_signal.process()
        print('---')

    time_final = int(np.round(time.time() * 1000))

    ffast.display_results(time_final - time_initial)

input freq: [80389]
true singleton bins:
    stage 0 bin 127
    stage 1 bin 247
found a singleton at 80389 -- stage 1 -- bin 247
---
input freq: [37106]
true singleton bins:
    stage 0 bin 62
    stage 1 bin 284
found a singleton at 37106 -- stage 1 -- bin 284
---
input freq: [101508]
true singleton bins:
    stage 0 bin 323
    stage 1 bin 67
found a singleton at 101508 -- stage 1 -- bin 67
---
input freq: [111452]
true singleton bins:
    stage 0 bin 320
    stage 1 bin 264
found a singleton at 111452 -- stage 0 -- bin 320
---
input freq: [66075]
true singleton bins:
    stage 0 bin 219
    stage 1 bin 12
---
input freq: [14932]
true singleton bins:
    stage 0 bin 183
    stage 1 bin 131
found a singleton at 14932 -- stage 0 -- bin 183
---
input freq: [26561]
true singleton bins:
    stage 0 bin 150
    stage 1 bin 208
---
input freq: [35083]
true singleton bins:
    stage 0 bin 97
    stage 1 bin 66
found a singleton at 35083 -- stage 0 -- bin 97
---
input freq: [47724]
true sing

In [47]:
output_signal.statistics()[0]/params.iterations

0.8

In [48]:
asdfasdf

NameError: name 'asdfasdf' is not defined

# this part is for debugging

In [ ]:
vars(ffast.config)

In [ ]:
def test_many(f, num_iter, **kwargs):
    r = []
    for i in tqdm(range(num_iter)):
        r.append(f(**kwargs))
    return r

In [ ]:
def test_if_location_is_true(stage_idx):
    input_signal.process()
    ffast.frontend.process()
    singleton_bin = input_signal.freqs[0] % params.bins[stage_idx]
    binprocessor = BinProcessor(config, ffast.frontend.delays, ffast.frontend.observation_matrix)
    
    binprocessor.adjust_to(config.bin_offsets[stage_idx]+singleton_bin, singleton_bin, stage_idx)
    binprocessor.process()
    loc_hat = binprocessor.location
    loc = input_signal.freqs[0]
    if loc_hat == loc:
        return True
    else:
        return False

In [ ]:
results_of_location_estimation = test_many(test_if_location_is_true, 100, stage_idx=0)
print('location identification accuracy: {}'.format(np.mean(results_of_location_estimation)))

In [ ]:
input_signal.process() # ffast.get_delays() in the CPP code as an argument
print("true frequencies: {}".format(input_signal.freqs))

ffast.frontend.process()
# ffast.backend.process()
# output_signal.process()

In [ ]:
# plt.stem(np.linalg.norm(ffast.frontend.observation_matrix, axis=0)**2/ffast.config.delays_nb)
# plt.xlabel('bin absolute index')
# plt.show()

In [ ]:
singleton_bin = input_signal.freqs[0] % params.bins[0]
print(singleton_bin)

In [ ]:
binprocessor = BinProcessor(config, ffast.frontend.delays, ffast.frontend.observation_matrix)

In [ ]:
# plt.stem(np.linalg.norm(ffast.frontend.observation_matrix, axis=0)**2/ffast.config.delays_nb)
# plt.axhline(binprocessor.thresholds[0])
# plt.xlabel('bin absolute index')
# plt.show()

In [ ]:
binprocessor.adjust_to(singleton_bin, singleton_bin, 0)

In [ ]:
binprocessor.is_zeroton()

In [ ]:
binprocessor.process()

In [ ]:
# binprocessor.find_location_new()
print(binprocessor.location)
print(input_signal.freqs)

In [ ]:
binprocessor.is_singleton()

In [ ]:
binprocessor.location

In [ ]:
binprocessor.amplitude

In [ ]:
binprocessor.noise

In [ ]:
binprocessor.thresholds[0]

In [ ]:
binprocessor.minimum_energy

In [ ]:
ffast.backend.decoded_frequencies

In [ ]:
output_signal.check_full_recovery()

In [ ]:
output_signal.statistics()

In [ ]:
ffast = FFAST(config, input_signal, output_signal)

In [ ]:
input_signal.process() # ffast.get_delays() in the CPP code as an argument
print(input_signal.freqs)

In [ ]:
ffast.process()

In [ ]:
output_signal.process()

In [ ]:
output_signal.statistics()